In [54]:
from urllib.request import urlopen  
from tqdm import tqdm
from lxml import html
from datetime import datetime
from collections import Counter
import json
import requests
import time
import re
# from screeninfo import get_monitors
import pandas as pd
import random

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

# US

In [55]:
# followers_US = pd.read_csv('Influencers_US_final.csv')
# followers_US.head()

followers_US = []
with open('followers_US.txt', 'r') as f:
    for line in f:
        if "@" in line:
            follower = line.strip()[1:]
            print(follower)
            followers_US.append(follower)
f.close()

followers_US


RevBlackNetwork
TheDemocrats
msnbc
FoxNews
TheAtlantic
aldotcom
JudiciaryGOP
MSNBC
NBCNews
RandPaul
politico
joncoopertweets
HomelandDems
HouseJudiciary
RepSwalwell
NPR
CNN
LPMisesCaucus
RealSpikeCohen
YATPOfficial
justinamash
tedlieu
LPNational
ToddHagopian
theintercept


['RevBlackNetwork',
 'TheDemocrats',
 'msnbc',
 'FoxNews',
 'TheAtlantic',
 'aldotcom',
 'JudiciaryGOP',
 'MSNBC',
 'NBCNews',
 'RandPaul',
 'politico',
 'joncoopertweets',
 'HomelandDems',
 'HouseJudiciary',
 'RepSwalwell',
 'NPR',
 'CNN',
 'LPMisesCaucus',
 'RealSpikeCohen',
 'YATPOfficial',
 'justinamash',
 'tedlieu',
 'LPNational',
 'ToddHagopian',
 'theintercept']

In [56]:
# followers_US_list = followers_US.Influencer_Xhandle.tolist()

# count = Counter(followers_US)

# followers_US = []

# for key, value in count.items():
#     if value > 2:
#         followers_US.append(key)
        
# followers_US

In [57]:
tweets_dict_US = {}

In [58]:
driver = webdriver.Chrome()
driver.get('https://twitter.com/i/flow/login')
wait = WebDriverWait(driver, 10)


In [59]:
### Log in

# Username
username = wait.until(
    EC.presence_of_element_located((By.CSS_SELECTOR, 'input[autocomplete=username]'))
)
username.send_keys("rigautam@udel.edu")
time.sleep(3)
login_button = wait.until(
    EC.presence_of_element_located((By.CSS_SELECTOR, '[role=button].r-13qz1uu'))
)
login_button.click()
time.sleep(3)

try: 
    # Password
    password = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '[type=password]'))
    )
    password.send_keys("Sus1234!")
    time.sleep(3)
    login_button = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '[data-testid*=Login_Button]'))
    )
    login_button.click()
    time.sleep(3)
    
except: # if we get an error
    username = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "[data-testid*=ocfEnterTextTextInput]"))
    )
    username.send_keys("RichaGotem")
    time.sleep(3)
    
    next_button = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "[data-testid*=ocfEnterTextNextButton]"))
    )
    next_button.click()
    time.sleep(3)
    
    password = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "[name*=password]"))
    )
    password.send_keys("Sus1234!")
    time.sleep(3)
    
    login_button = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "[data-testid*=LoginForm_Login_Button]"))
    )
    login_button.click()
    time.sleep(3)
    

In [69]:
# Function to check if the Fetch More button is present
def is_element_present(driver, xpath):
    try:
        element = driver.find_element(By.XPATH, xpath)
        return element.is_displayed()
    except NoSuchElementException:
        return False

count = 0

keyword1 = "(Gaza* OR Israel* OR (West OR AND OR Bank) OR (Tel OR AND OR Aviv) OR (Tel OR AND OR Aviv-Yafo) OR Bedouin* OR Palestin* OR Middle OR East* OR Middle OR Eastern OR Jew* OR rabbi OR Muslim* OR Islam* OR Jihad OR Antisemiti* OR Zionis* OR IDF)"
keyword2 = "(IOF OR Hamas OR Massacre OR Genocide OR Ceasefire OR Terroris* OR Netanyahu OR Histadrut OR Haniyeh OR Yahya OR Sinwar OR Fatah OR Mohammed OR Deif)"
hashtags = "(# OR #IsraelPalestineWar OR #IsraelGazaWar OR #Gazabombing OR #CeasefireNOW OR #FreePalestine OR #StrikeForGaza OR #Gazagenocide)"

for profile in tqdm(followers_US):
    
    if profile not in tweets_dict_US.keys():
        print(profile)

        tweets_dict_US[profile] = []

        ### Begin scrape
        time.sleep(3)

        # Search
        search_button = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '[aria-label="Search and explore"]'))
        )
        search_button.click()
        time.sleep(3)

        search_bar = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '[aria-label="Search"]'))
        )
        search_bar.click()
        time.sleep(3)

        for i in range(2): 
            search_query = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '[aria-label="Search query"]')))

            # Advanced search query
            if i == 0:
                search_query.send_keys(f'{keyword1} {hashtags} (from:{profile}) min_faves:1000 until:2024-09-14 since:2023-10-07 -filter:links -filter:replies')
            if i == 1:
                search_query.send_keys([Keys.BACKSPACE]*1000)
                search_query.send_keys(f'{keyword2} {hashtags} (from:{profile}) min_faves:1000 until:2024-09-14 since:2023-10-07 -filter:links -filter:replies')

            search_query.send_keys(Keys.RETURN)
            time.sleep(6)
            
            # Scroll until you cannot scroll anymore
            reached_page_end = False
            last_height = driver.execute_script("return document.body.scrollHeight")

            retry_xpath = "//button[@role='button' and @type='button']//span[text()='Retry']"
            # random.randrange(10, 15)
            if (is_element_present(driver, retry_xpath)):
                for i in range(5):
                    search_button = wait.until(EC.element_to_be_clickable((By.XPATH, retry_xpath)))
                    search_button.click()
                    print(f'{profile} - {count} - Retry button appeared.')
                time.sleep(10) 

            while not reached_page_end:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")   
                time.sleep(3)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if last_height == new_height:
                    reached_page_end = True
                else:
                    last_height = new_height
            try:
                # Select the home timeline div
                search_results = driver.find_element(By.CSS_SELECTOR, '[aria-label="Timeline: Search timeline"]')
                # Just get all the HTML, will parse later
                tweets_listed = search_results.get_attribute('innerHTML')
                time.sleep(3)

                # Append to dict
                tweets_dict_US[profile].append(tweets_listed)

            except:
                pass

    count+=1



  0%|          | 0/25 [00:00<?, ?it/s]

RealSpikeCohen


 76%|███████▌  | 19/25 [01:13<00:23,  3.85s/it]

YATPOfficial


 80%|████████  | 20/25 [02:15<00:39,  7.94s/it]

...

 92%|█████████▏| 23/25 [05:57<00:55, 27.95s/it]

ToddHagopian


 96%|█████████▌| 24/25 [07:05<00:34, 34.66s/it]

theintercept


100%|██████████| 25/25 [08:30<00:00, 20.41s/it]


In [70]:
driver.quit()

In [71]:
tweets_dict_US.keys()

dict_keys(['RevBlackNetwork', 'TheDemocrats', 'msnbc', 'FoxNews', 'TheAtlantic', 'aldotcom', 'JudiciaryGOP', 'MSNBC', 'NBCNews', 'RandPaul', 'politico', 'joncoopertweets', 'HomelandDems', 'HouseJudiciary', 'RepSwalwell', 'NPR', 'CNN', 'LPMisesCaucus', 'RealSpikeCohen', 'YATPOfficial', 'justinamash', 'tedlieu', 'LPNational', 'ToddHagopian', 'theintercept'])

In [72]:
tweets_dict_US_cleaned = {}
    
for key, value in tweets_dict_US.items():
    if key not in tweets_dict_US_cleaned.keys():
        tweets_dict_US_cleaned[key] = []
        for v in value:
            soup = BeautifulSoup(v, 'html.parser')
            tweets = soup.find_all(attrs={"data-testid": "tweetText"})
            for tweet in tweets:
                tweet_text = tweet.get_text(strip=True)
                tweets_dict_US_cleaned[key].append(tweet_text)

tweets_dict_US_cleaned.keys()

dict_keys(['RevBlackNetwork', 'TheDemocrats', 'msnbc', 'FoxNews', 'TheAtlantic', 'aldotcom', 'JudiciaryGOP', 'MSNBC', 'NBCNews', 'RandPaul', 'politico', 'joncoopertweets', 'HomelandDems', 'HouseJudiciary', 'RepSwalwell', 'NPR', 'CNN', 'LPMisesCaucus', 'RealSpikeCohen', 'YATPOfficial', 'justinamash', 'tedlieu', 'LPNational', 'ToddHagopian', 'theintercept'])

In [73]:
with open("US_tweets.json", 'w') as file:
    json.dump(tweets_dict_US, file)

In [74]:
with open("US_tweets_cleaned.json", 'w') as file:
    json.dump(tweets_dict_US_cleaned, file)